In [1]:
%run stdPackages.ipynb # this imports a lot of useful packages

#%run/stdPackages.ipynb
#test2
#tes test

In [2]:
import base
import os 

def read_csv_with_path(directory_path, file_name):
    # Use os.path.join to create the full file path
    file_path = os.path.join(directory_path, file_name)
    
    # Use the read_csv function to load the data into a pandas DataFrame
    df = pd.read_csv(file_path)
    
    return df

Data og parametre

In [18]:
#Laver vores database
db = pyDbs.SimpleDB()
#Tilføjer parametre
db['MWP'] = 50

Downloading data: Load data

In [14]:
#Denne linjer danner stien til vores working directory
curr = os.getcwd()
#Denne koder henter csv-filen
HourlyLoad_df = read_csv_with_path(curr,"Total_load.csv")
#Fjerner potentielle manglende observationer
HourlyLoad_df = HourlyLoad_df.dropna()
#Laver en variabel for hver time på en døgn ud fra dato værdien
HourlyLoad_df['h_id'] = HourlyLoad_df['Time (CET/CEST)'].str[11:16]
#Laver et index fra 1-24
Index_h = pd.Index(list(range(1,25)), name = 'h')
#Udregner gennemsnitlig load pr. time ud fra hele året
Average_hourly_load = pd.Series(HourlyLoad_df.groupby('h_id')['Actual Total Load [MW] - BZN|DK2'].mean(), name = 'Average Hourly Load')
#Navngiver indexet 1-24 i stedet for 00:00-23:00
Average_hourly_load = Average_hourly_load.rename(index=dict(zip(Average_hourly_load.index, Index_h)))
#Laver et dobbelt index. Vi antager en forbruger.
Average_hourly_load.index = pd.MultiIndex.from_product([Average_hourly_load.index, ['c1']], names=['h', 'consumer'])

In [15]:
Average_hourly_load

h   consumer
1   c1          1251.202740
2   c1          1201.383562
3   c1          1169.797260
4   c1          1157.789041
5   c1          1158.621918
6   c1          1210.189041
7   c1          1356.353425
8   c1          1514.895890
9   c1          1611.136986
10  c1          1660.983562
11  c1          1692.493151
12  c1          1690.397260
13  c1          1683.194521
14  c1          1672.112329
15  c1          1649.980822
16  c1          1628.378082
17  c1          1657.315068
18  c1          1747.443836
19  c1          1736.723288
20  c1          1646.265753
21  c1          1560.821918
22  c1          1482.054795
23  c1          1400.153425
24  c1          1316.602740
Name: Average Hourly Load, dtype: float64

In [17]:
#Tilføjer databasen
db['Max hourly load'] = Average_hourly_load
db['Max hourly load']

h   consumer
1   c1          1251.202740
2   c1          1201.383562
3   c1          1169.797260
4   c1          1157.789041
5   c1          1158.621918
6   c1          1210.189041
7   c1          1356.353425
8   c1          1514.895890
9   c1          1611.136986
10  c1          1660.983562
11  c1          1692.493151
12  c1          1690.397260
13  c1          1683.194521
14  c1          1672.112329
15  c1          1649.980822
16  c1          1628.378082
17  c1          1657.315068
18  c1          1747.443836
19  c1          1736.723288
20  c1          1646.265753
21  c1          1560.821918
22  c1          1482.054795
23  c1          1400.153425
24  c1          1316.602740
Name: Average Hourly Load, dtype: float64